In [16]:
from PIL import Image
import numpy as np
import os 
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
import torch
from torch import nn
import sys
from sklearn.model_selection import train_test_split
sys.path.append('../codes/')
from network import CNN_with_decoder,CNN2,Decoder
from losses import  loss
from optimizer import create_optimizer
from data import ImageData
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 定义模型

In [6]:
fp=open('../logs/output.log','a+')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN2().to(device)
# 定义代价函数
criterion = torch.nn.CrossEntropyLoss()

# 定义优化器
optimizer = torch.optim.Adam(model.parameters())

epochs = 10

# 读取数据

In [31]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
])


In [58]:
dataset = ImageData('../data_base_500/train')
train_set, val_set = train_test_split(dataset, test_size=0.3, random_state=42)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=128, shuffle=False)

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
demo_model = CNN_with_decoder()

# Train Decoder

In [78]:
# load model weights
pretrained_dict  = torch.load('../models/Finance_cnn2_lr1e-05_adam_wd0.0001_epoch30_model.pth')

my_model = Decoder()
new_model_dict = my_model.state_dict()
for key in pretrained_dict:
    if key in new_model_dict:
        new_model_dict[key] = pretrained_dict[key]
my_model.load_state_dict(new_model_dict)

<All keys matched successfully>

In [36]:
# 冻结CNN部分
for param in my_model.parameters():
    param.requires_grad = False
for param in my_model.decoder.parameters():
    param.requires_grad = True

In [37]:
# load loss function
from losses import CustomLoss
from engine import train_decoder_model
optimizer = torch.optim.Adam(my_model.decoder.parameters(), lr=0.001)
loss = nn.MSELoss()


In [70]:

my_model.to(device)
model = train_decoder_model(my_model, train_loader, val_loader,
                            optimizer,10,loss,device)

Start training...
Epoch [1/10] - Train Loss: 3.1398 - Val Loss: 24.6036
Val R square: 0: 0.9181, 1: 0.8711, 2: 0.8034
Epoch [2/10] - Train Loss: 3.1181 - Val Loss: 27.7167
Val R square: 0: 0.9112, 1: 0.8647, 2: 0.7680
Epoch [3/10] - Train Loss: 2.8681 - Val Loss: 25.1781
Val R square: 0: 0.9176, 1: 0.8463, 2: 0.8069
Epoch [4/10] - Train Loss: 2.3175 - Val Loss: 25.8073
Val R square: 0: 0.9157, 1: 0.8636, 2: 0.7915
Epoch [5/10] - Train Loss: 3.2135 - Val Loss: 24.3413
Val R square: 0: 0.9181, 1: 0.8781, 2: 0.8043
Epoch [6/10] - Train Loss: 1.6700 - Val Loss: 25.3223
Val R square: 0: 0.9166, 1: 0.8581, 2: 0.8005
Epoch [7/10] - Train Loss: 2.6898 - Val Loss: 28.2150
Val R square: 0: 0.9102, 1: 0.8719, 2: 0.7580
Epoch [8/10] - Train Loss: 3.0910 - Val Loss: 24.3203
Val R square: 0: 0.9177, 1: 0.8782, 2: 0.8053
Epoch [9/10] - Train Loss: 1.7176 - Val Loss: 25.1125
Val R square: 0: 0.9174, 1: 0.8711, 2: 0.7964
Epoch [10/10] - Train Loss: 2.7043 - Val Loss: 27.1145
Val R square: 0: 0.9094, 1:

In [80]:
my_model.to(device)
predict_data = [[] for _ in range(3)]
origin_data = [[] for _ in range(3)]
with torch.no_grad():
    for inputs, _, attr in val_loader:
        inputs, attr = inputs.to(device), attr.to(device)
        outputs = my_model(inputs)
        for i in range(3):
            predict_data[i].extend(list(outputs[:, i].cpu().numpy()))
            origin_data[i].extend(list(attr[:, i].cpu().numpy()))


In [81]:
from sklearn.metrics import r2_score
r2_score(predict_data[0],origin_data[0])

-1436498.730761146

# Train Self-supervised Model